# Reimplementing a Choice-Based Revenue Management Simulation

We reimplement Simulation 2 from "Revenue Management Under a General Discrete Choice Model of Consumer Behavior" by Talluri and van Ryzin 2004. 

In [1]:
import numpy as np
import scipy.stats as stats

from data import REVENUES
from choice import choice_dp_tabulate, mnl
from simulation import simulate, calculate_statistics, confidence_interval, precompute_emsr_assortments

np.random.seed(0)  # For reproducibility

In [2]:
# Problem Constants
C = 185
T = 410
mean_demand = 205
n = len(REVENUES)
arrival_rate = 0.5
# high is -0.005, low is -0.0015, we will calculate a range of betas
beta = -0.0015
U = np.array(REVENUES, dtype=float) * beta
w = np.exp(U)

In [3]:
# Implement Choice by solving DP
value_fcn, action_fcn = choice_dp_tabulate(arrival_rate, C, T, list(REVENUES), list(U), mnl)
# Implement EMSR by precomputing for each x, t remaining
emsr_assort = precompute_emsr_assortments(C, T, mean_demand, n)

In [4]:
replications = 1000
average_esmr_load_factor = []
average_mnl_load_factor = []
average_esmr_revenue = []
average_mnl_revenue = []

for i in range(replications):
    esmr_purchases, mnl_purchases = simulate(T, C, arrival_rate, w, emsr_assort, action_fcn)
    esmr_load_factor, esmr_revenue = calculate_statistics(esmr_purchases, C)
    mnl_load_factor, mnl_revenue = calculate_statistics(mnl_purchases, C)

    average_esmr_load_factor.append(esmr_load_factor)
    average_mnl_load_factor.append(mnl_load_factor)
    average_esmr_revenue.append(esmr_revenue)
    average_mnl_revenue.append(mnl_revenue)

print("Average EMSR Load Factor:", confidence_interval(average_esmr_load_factor))
print("Average MNL Load Factor:", confidence_interval(average_mnl_load_factor))
print("Average EMSR Revenue:", confidence_interval(average_esmr_revenue))
print("Average MNL Revenue:", confidence_interval(average_mnl_revenue))

Average EMSR Load Factor: (0.8451793425681697, 0.8515449817561547)
Average MNL Load Factor: (0.7160225057672059, 0.7244639807192804)
Average EMSR Revenue: (57776.10634791489, 58386.66365208512)
Average MNL Revenue: (65925.25467707166, 66719.84532292835)
